In [ ]:
%matplotlib inline
import pandas as pd
from pandas.plotting import table
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

In [ ]:
#PROJFILE =  ['projects.csv', 'anotherpath/projects.csv']
PROJFILE = ['/tmp/zfssa_explorer_XXX.XXX.XXX.XXX_XXXXXX_XXXXXX/projects.csv']

Just a function to change sizes to human readable for return values

In [ ]:
suffixes = ['B', 'KB', 'MB', 'GB', 'TB', 'PB']
def human_size(nbytes):
    i = 0
    while nbytes >= 1024 and i < len(suffixes)-1:
        nbytes /= 1024.
        i += 1
    f = ('%.2f' % nbytes).rstrip('0').rstrip('.')
    return '%s %s' % (f, suffixes[i])

In [ ]:
#dataraw = pd.read_csv(PROJFILE, header=1, sep=";")
dataraw = ""
if len(PROJFILE) == 1:
    dataraw = pd.read_csv(str(PROJFILE).strip("[']"), header=1, sep=";")
else:
    filelist = []
    for file in PROJFILE:
        x = 0
        dfname = "df{}".format(x)
        dfname = pd.read_csv(file, header=1, sep=";")
        filelist.append(dfname)
        x + 1
    dataraw = pd.concat(filelist)

Show available columns

In [ ]:
dataraw.columns

In [ ]:
countproj = dataraw['id'].count()
display(Markdown("#### Projects count: {}".format(countproj)))

### Get projects per pool

In [ ]:
projpool = dataraw[['name', 'pool']].rename(columns={'name': 'projects'}).groupby('pool').count()

plt.figure(figsize=(12, 4))
# plot bar
ax1 = plt.subplot(121)
ax1 = projpool.plot(kind='bar', legend=False, ax=ax1, fontsize=12, grid=True)
ax1.set_ylabel('Projects')
ax1.set_xlabel('pool')
# plot table
ax2 = plt.subplot(122)
plt.axis('off')
tbl = table(ax2, projpool, loc='center', bbox=[0.2, 0.2, 0.5, 0.5])
tbl.auto_set_font_size(False)
tbl.set_fontsize(14)
plt.show()


### Check for duplicated projects in different pools

In [ ]:
projpool = dataraw[['name', 'pool']]
nonunique = projpool[projpool['name'].duplicated()].groupby('name')
dup = projpool[projpool.name.isin(nonunique.name.all())]
dup.set_index('name').sort_index()

### Usage

In [ ]:
sumspace = dataraw[['space_total', 'space_data','space_unused_res', 'space_unused_res_shares',
                    'reservation']].sum()
sumspace.map(human_size)

### Usage per pool

In [ ]:
spacepool = dataraw[['space_total', 'space_data','space_unused_res', 'space_unused_res_shares',
                     'reservation', 'pool']].groupby('pool')
spacepool.sum().applymap(human_size)

### Get projects using compression

In [ ]:
compress = dataraw[['name', 'pool', 'compressratio']]
compress[compress['compressratio'] != 100]